In [42]:
import polars as pl

movies_pl = pl.scan_csv('movies.csv')
ratings_pl = pl.scan_csv('ratings.csv')
tags_pl = pl.scan_csv('tags.csv')


In [5]:
ratings_pl = ratings_pl.with_columns(
    pl.from_epoch(pl.col('timestamp'), time_unit='s').alias('datetime_timestamp')
).with_columns(
    pl.col('datetime_timestamp').dt.year().alias('year'),
    pl.col('datetime_timestamp').dt.month().alias('month')
).lazy()


In [7]:
# a. Show the aggregated number of ratings per year

ratings_by_year = ratings_pl.group_by('year').agg(
    pl.count('rating').alias('# ratings')
).sort('year', descending=True)

ratings_by_year.collect().head(20)

year,# ratings
i32,u32
2018,6418
2017,8198
2016,6703
2015,6616
2014,1439
…,…
2003,4014
2002,3478
2001,3922


In [8]:
# b. Show the average monthly number of ratings

avg_monthly = ratings_pl.group_by('month', 'year').agg(
    pl.mean('rating').alias('# ratings')
).sort(['year', 'month'], descending=True)

avg_monthly.collect().head(20)


month,year,# ratings
i8,i32,f64
9,2018,3.568709
8,2018,3.557762
7,2018,4.010239
6,2018,3.979714
5,2018,2.95163
…,…,…
6,2017,2.959424
5,2017,3.480184
4,2017,3.626219


In [117]:
# c. Show the rating levels distribution

ratings_pl.with_columns(
    pl.when(pl.col('rating').is_between(0, 2))
    .then(pl.lit('0-2'))
    .when(pl.col('rating').is_between(2.3, 4))
    .then(pl.lit('2.5-4'))
    .otherwise(pl.lit('>4'))
    .alias('rating_bucket')
).select(
  'rating_bucket',
  pl.count('rating').over('rating_bucket').alias('count')
).unique().sort('rating_bucket').collect()


rating_bucket,count
str,u32
"""0-2""",13523
"""2.5-4""",65551
""">4""",21762


In [108]:
df = pl.DataFrame({"foo": [1, 3, 4], "bar": [3, 4, 0]}).lazy()

df.with_columns(pl.when(pl.col("foo") > 2).then(1).otherwise(-1).alias("val")).collect()

foo,bar,val
i64,i64,i32
1,3,-1
3,4,1
4,0,1


In [9]:
# d. Show the 18 movies that are tagged but not rated

tags_pl_joined = tags_pl.join(ratings_pl, on='movieId', how='left').filter(pl.col('rating').is_null())
tags_pl_joined.join(movies_pl, on='movieId', how='inner').select(pl.col('title')).unique().sort('title').collect()

title
str
"""Browning Version, The (1951)"""
"""Call Northside 777 (1948)"""
"""Chalet Girl (2011)"""
"""Chosen, The (1981)"""
"""Color of Paradise, The (Rang-e…"
…
"""Road Home, The (Wo de fu qin m…"
"""Roaring Twenties, The (1939)"""
"""Scrooge (1970)"""


In [10]:
# e. Show the movies that have rating but no tag

#.select('movieId').unique()
ratings_pl_joined = ratings_pl.join(tags_pl, on='movieId', how='left').filter(pl.col('tag').is_null())
ratings_pl_joined.join(movies_pl, on='movieId', how='inner').select(pl.col('title')).unique().sort('title').collect().head(20)

title
str
"""'71 (2014)"""
"""'Hellboy': The Seeds of Creati…"
"""'Round Midnight (1986)"""
"""'Salem's Lot (2004)"""
"""'Til There Was You (1997)"""
…
"""10 Years (2011)"""
"""10,000 BC (2008)"""
"""100 Girls (2000)"""


In [11]:
# f. Focusing on the rated untagged movies with more than 30 user ratings,show the top 10 movies in terms of average rating and number of ratings

rated_untagged = ratings_pl_joined.group_by('movieId').agg(
    pl.count('rating').alias('# ratings'),
    pl.mean('rating').alias('avg_ratings')
).filter(
    pl.col('# ratings') > 30
)

rated_untagged = rated_untagged.join(movies_pl, on='movieId', how='inner').select(pl.col('title'), pl.col('avg_ratings'), pl.col('# ratings'))
rated_untagged.sort('avg_ratings', descending=True).collect().head(10)
rated_untagged.sort('# ratings', descending=True).collect().head(10)


title,avg_ratings,# ratings
str,f64,u32
"""American Beauty (1999)""",4.056373,204
"""Ace Ventura: Pet Detective (19…",3.040373,161
"""Mask, The (1994)""",3.184713,157
"""Die Hard (1988)""",3.862069,145
"""Die Hard: With a Vengeance (19…",3.555556,144
"""Groundhog Day (1993)""",3.944056,143
"""Dumb & Dumber (Dumb and Dumber…",3.06015,133
"""GoldenEye (1995)""",3.496212,132
"""Monsters, Inc. (2001)""",3.871212,132


In [13]:
# g. Identify the users that tagged movies without rating them

tags_pl_joined = tags_pl.join(ratings_pl, on='movieId', how='left').filter(pl.col('rating').is_null())
tags_pl_joined.select(pl.col('userId')).unique().collect()

userId
i64
318
474
543
288


In [14]:
# h. What is the average number of ratings per user in ratings DF? And the average number of ratings per movie?

ratings_pl = ratings_pl.filter(pl.col('rating').is_not_null())

ratings_pl.select(
    'userId',
    pl.col('rating').count().alias('# ratings'),
    pl.col('userId').unique().count().alias('# users')
).select(
    (pl.col('# ratings') / pl.col('# users')).round(3).alias('avg_no_of_ratings_per_user')
).collect()

ratings_pl.select(
    'userId',
    pl.col('rating').count().alias('# ratings'),
    pl.col('movieId').unique().count().alias('# users')
).select(
    (pl.col('# ratings') / pl.col('# users')).round(3).alias('avg_no_of_ratings_per_user')
).collect()


avg_no_of_ratings_per_user
f64
10.37


In [96]:
# i. What is the predominant (frequency based) genre per rating level?

ratings_pl.join(movies_pl, on='movieId', how='left').group_by(['genres', 'rating']).agg(
    pl.len().alias("counts")
).with_columns(
    pl.col('counts')
    .rank('dense', descending=True)
    .over('rating')
    .alias("rank")
).sort('rating', descending=True).filter(
    pl.col('rank') == 1
).collect()


genres,rating,counts,rank
str,f64,u32,u32
"""Drama""",5.0,895,1
"""Drama""",4.5,593,1
"""Drama""",4.0,2055,1
"""Comedy""",3.5,854,1
"""Comedy""",3.0,1614,1
"""Comedy""",2.5,515,1
"""Comedy""",2.0,828,1
"""Comedy""",1.5,256,1
"""Comedy""",1.0,348,1


In [95]:
# j. Predominant tag per genre

joined_df = movies_pl.join(tags_pl, on='movieId', how='left')
joined_df.group_by(['genres', 'tag']).agg(
    pl.len().alias("counts")
).with_columns(
    pl.col('counts')
    .rank('dense', descending=True)
    .over('genres')
    .alias("rank")
).sort('genres', descending=True).filter(
    pl.col('rank') == 1
).collect()


genres,tag,counts,rank
str,str,u32,u32
"""Western""",null,21,1
"""War""",null,4,1
"""Thriller""",null,75,1
"""Sci-Fi|Thriller|IMAX""",null,1,1
"""Sci-Fi|Thriller""",null,22,1
…,…,…,…
"""Action|Adventure|Animation|Chi…",null,6,1
"""Action|Adventure|Animation""",null,18,1
"""Action|Adventure""",null,29,1


In [52]:
# k. Top 10 popular movies (most users seen/rated it)

ratings_pl.join(movies_pl, left_on="movieId", right_on="movieId", how="left").unique().select(
        'title',
        pl.count('userId').over('title').alias('counts')
    ).unique().with_columns(
      rn = pl.col('counts').rank('dense', descending=True)
    ).filter(
      pl.col('rn') <= 10
).sort('rn').collect()


title,counts,rn
str,u32,u32
"""Forrest Gump (1994)""",329,1
"""Shawshank Redemption, The (199…",317,2
"""Pulp Fiction (1994)""",307,3
"""Silence of the Lambs, The (199…",279,4
"""Matrix, The (1999)""",278,5
"""Star Wars: Episode IV - A New …",251,6
"""Jurassic Park (1993)""",238,7
"""Braveheart (1995)""",237,8
"""Terminator 2: Judgment Day (19…",224,9


In [97]:
# l. Top 10 movies in terms of avg rating (>30 users reviewed)

ratings_pl.join(movies_pl, on='movieId', how='left').select(
    'title',
    pl.col('userId').unique().count().over('movieId').alias('counts'),
    pl.mean('rating').over('movieId').alias('avg_rating')
).filter(
  pl.col('counts') > 30 
).unique().with_columns(
    rn = pl.col('avg_rating').rank('dense', descending=True)
    ).filter(
      pl.col('rn') <= 10
).sort('rn').collect()


title,counts,avg_rating,rn
str,u32,f64,u32
"""Shawshank Redemption, The (199…",317,4.429022,1
"""Lawrence of Arabia (1962)""",45,4.3,2
"""Godfather, The (1972)""",192,4.2890625,3
"""Fight Club (1999)""",218,4.272936,4
"""Cool Hand Luke (1967)""",57,4.27193,5
"""Dr. Strangelove or: How I Lear…",97,4.268041,6
"""Rear Window (1954)""",84,4.261905,7
"""Godfather: Part II, The (1974)""",129,4.25969,8
"""Departed, The (2006)""",107,4.252336,9
